In [1]:
import os
import numpy as np
from xml.dom import minidom
from xml.dom.minidom import parse
from TAB_generator import *

In [129]:
class TABinfo:
    def __init__(self, beat=4, beat_type=4, capo=0, tuning=[0,0,0,0,0,0], original=None, name=None):
        self.beat = beat
        self.beat_type = beat_type
        self.capo = capo
        self.tuning = tuning
        self.original = original
        self.name = name
    def showinfo(self):
        infodict = {
            'beat': int(self.beat),
            'beat_type': int(self.beat_type),
            'capo': self.capo,
            'tuning': self.tuning,
            'original': self.original,
            'name': self.name
        }
        print(infodict)
class Tablature:
    def __init__(self, pitch, finger, time, info=None):
        self.pitch = pitch #该歌曲的音符，每个小节为一个list，所有小节组合成一个大list
        self.finger = finger #指法
        self.time = time #时值
        self.info = info
    def choose_measure(self, measurenum):
        '''
        check the given measure number()
        =============================
        measurenum: int or list
        the measure number you want to check
        '''
        if isinstance(measurenum, int):
            measure = Tablature(pitch=self.pitch[measurenum],
                                finger=self.finger[measurenum],
                                time=self.time[measurenum],
                                info=self.info
                               )
            return measure
        elif isinstance(measurenum, (list, tuple)):
            measure = Tablature(pitch=self.pitch[measurenum[0]:measurenum[1]],
                                finger=self.finger[measurenum[0]:measurenum[1]],
                                time=self.time[measurenum[0]:measurenum[1]],
                                info = self.info)
            return measure
        else:
            raise Exception('Input Error: input should be int or list, but {}'.format(measurenum, type(measurenum)))
    def vectorization(self, division=16):
        measurenum = len(self.pitch)
        measurelist = []
        for i in range(measurenum):
            measurelist.append(self.choose_measure(i))
        
        pitchvec, fingervec = [], []
        for measure in measurelist:
            info = measure.info #
            beat = info.beat
            pitchseries, fingerseries= [], []
            for i, index in enumerate(zip(measure.pitch, measure.finger, measure.time)):
                note = []
                finger = []
                for i in range(int(index[2] * (division/4))):
                    if i == 0:
                        note.append(index[0])
                        finger.append(index[1])
                    else:
                        note.append('__')
                        finger.append('__')

                for notezip in zip(note, finger):
                    pitchseries.append(notezip[0])
                    fingerseries.append(notezip[1])
            pitchvec.append(pitchseries)
            fingervec.append(fingerseries)
        vecsong = Tablature(pitch=pitchvec, finger=fingervec, time=self.time, info=self.info)
        return vecsong
    
    def root_and_melody(self, clear_zero=True):
        '''
        根音和旋律音检测
        measure:一个三个元素的list，分别是小节内的所有音、指法、时间
        '''
        
        measurenum = len(self.pitch)
        measurelist = []
        for i in range(measurenum):
            measurelist.append(self.choose_measure(i))
        
        readymeasures = []
        for measure in measurelist:
            
            pitchs, fingers, times = measure.pitch, measure.finger, measure.time #分别代表音高、指法、时间
            tune, capo = measure.info.tuning, measure.info.capo
            pitchnum = len(pitchs)
            indexlist = list(np.zeros(pitchnum))
            #print('待检测的音为{}'.format(pitchs))
            roots = [] 
            melodys = []
            root_thr = 5

            #设置根音门槛
            allfingers = []
            for finger in fingers:
                allfingers.extend(finger.split())
            rootcount = 0
            for finger in allfingers:
                if ('R' not in finger) and int(finger[1]) >= 5:
                    rootcount += 1
            if rootcount <= 1:
                root_thr = 4


            for i in range(pitchnum):  #该小节内的第i个音

                multi = 0
                pitch_cu = pitchs[i].split()
                finger_cu = fingers[i].split()
                if len(pitch_cu) >= 2:
                    multi = 1
                melody, root = 0, 0
                if 'R' not in finger_cu[0]:
                    for k in range(len(finger_cu)):
                        finger_cu[k] = midi2finger(finger2midi(finger_cu[k],tune=tune,capo=capo), tune=tune, capo=capo)[-1]
                    #print('note {}: {}, thr={}'.format(i, finger_cu, root_thr))
                    if (int(finger_cu[0][1]) <= 3) or (multi == 1) :  #当音簇最高音在1或2弦，或者音簇为多音时
                        #melody = finger2midi(finger_cu[0])  
                        indexlist[i] += 10
                        melody = str2midi(pitch_cu[0])
                        #print('    旋律音:{}, 其midi值为{}, 在{}'.format(midi2str(int(melody)), melody, finger_cu[0]))
                    if (int(finger_cu[-1][1]) >= root_thr):  #当音簇最低音在5或6弦时
                        root = str2midi(pitch_cu[-1]) 
                        indexlist[i] += 5
                        #print('    根音:{}, 其midi值{}, 在{}'.format(midi2str(int(root)), root, finger_cu[-1]))
                    #print('---------------')
                # roots.append(midi2str(int(root)))      
                # melodys.append(midi2str(int(melody)))
            #return [melodys, roots]  #返回该小节内所有满足条件的根音和旋律音

            for i in range(pitchnum):
                if indexlist[i] == 15:#当根音旋律音都有
                    pitchs[i] = pitchs[i].split()[0] + ' '+pitchs[i].split()[-1]
                    fingers[i] = fingers[i].split()[0] + ' '+fingers[i].split()[-1]
                elif indexlist[i] == 10:#只有旋律音
                    pitchs[i] = pitchs[i].split()[0]
                elif indexlist[i] == 5:#只有根音
                    fingers[i] = fingers[i].split()[-1]
            if indexlist[0] == 0:
                pitchs[0], fingers[0] = 'R', '(R,R)'
            for i in range(len(times)-1, -1, -1):
                if indexlist[i] == 0 and i >0:#当二者都没有时：
                    times[i-1] += times[i]
                    times[i] = 0 #将该时值交给前面
                    pitchs[i], fingers[i] = '', ''
            #最后清除空元素
            if clear_zero:
                for i in range(len(times)-1, -1, -1):
                    if times[i] == 0:
                        pitchs.pop(i)
                        fingers.pop(i)
                        times.pop(i)

            return1 = Tablature(pitch=pitchs, finger=fingers, time=times, info=self.info)
            readymeasures.append(return1)
            
        totalreturn = measure_join(readymeasures)  
        return totalreturn
    def showall(self):
        print('pitch: {}'.format(self.pitch))
        print('finger: {}'.format(self.finger))
        print('time: {}'.format(self.time))
        
def readTAB(path, pitchtype='default', show_processing=False, show_tuning=False):
    '''
    Read a TAB in musicXML form, return the pitch, finger position and note beat
    ========
    pitchtype: change the text order in tokenized note. For example 'default' for 5C#, and 'music21' for C#5
    tune: adjust for special tuning. For example, DADGAD tuning is [-2, 0, 0, 0, -2, -2]
    '''
    def check_repeat(measurelist):
            '''
            检查每个小节的反复记号与房子，并记录下来以便展开为时间序列
            '''
            repeatstart = '1'
            repeatend = '2'
            repeatmark = list(np.zeros(len(measurelist)))
            housemark = list(np.zeros(len(measurelist)))
            for num, measure in enumerate(measurelist):
                mark = ''
                if haveNodes(measure, 'ending'):
                    endingNode = measure.getElementsByTagName('ending')
                    endingnumber = endingNode[0].getAttribute('number').split(', ')
                    mark += ''.join(endingnumber)
                    for Node in endingNode:
                        if Node.getAttribute('type') == 'start':
                            mark += 'a'
                        if Node.getAttribute('type') == 'stop':
                            mark += 'b'
                            mark += Node.getAttribute('times')

                housemark[num] = mark   

            for num, measure in enumerate(measurelist):
                mark = ''    
                if haveNodes(measure, 'repeat'):
                    repeatNode = measure.getElementsByTagName('repeat')
                    for Node in repeatNode:
                        if Node.getAttribute('direction') == 'forward':
                            mark += 'a'
                        if Node.getAttribute('direction') == 'backward':
                            mark += 'b'
                            mark += Node.getAttribute('times')

                repeatmark[num] = mark
            return repeatmark, housemark
        
    def repeater(pitchs, repeatmark, housemark):
        '''
        Repeat the Da capo measures and process the different ending by given repeatmark
        '''
        if len([i for i in repeatmark if i != '']) == 0 and len([i for i in housemark if i != '']) == 0:
            return pitchs
        else:      
            play = []
            abpair = []
            housepair = []
            for i in range(len(repeatmark)):
                if repeatmark[i] == 'a':
                    start = i
                if 'b' in str(repeatmark[i]):
                    end = i
                    times = str(repeatmark[i])[str(repeatmark[i]).index('b')+1:]
                    abpair.append((start, end+1, int(times)))

            for i in range(len(housemark)):
                if 'a' in str(housemark[i]):
                    charge = int(housemark[i][:housemark[i].index('a')])

                    start = i
                if 'b' in str(housemark[i]):
                    end = i

                    housepair.append((start, end, charge))
            ready_for_repeat = pitchs[:abpair[0][0]]

            play = []
            for j in range(len(abpair)):
                repeattime = abpair[j][2]
                repeatunit = pitchs[abpair[j][0]:abpair[j][1]]
                frag_unit = repeatunit[:housepair[j][0]-abpair[j][0]]

                if j < len(abpair)-1:
                    singleunit = pitchs[abpair[j][1]:abpair[j+1][0]]
                else:
                    singleunit = pitchs[abpair[j][1]:]

                charge = housepair[j][-1]
                for i in range(repeattime):
                    if str(i+1) in str(charge):
                        for unit in repeatunit:
                            play.append(unit)
                    elif str(i+1) not in str(charge):
                        for unit in frag_unit:
                            play.append(unit)
                for unit in singleunit:
                    play.append(unit)
            return play
    if path[-2:] != '\\':
        path += '\\'
    XML_files = os.listdir(path)
    XML_files = [file for file in XML_files if '.xml' in file]

    TAB = []
    TABobjects = []
    for file_num, file in enumerate(XML_files):  
        print('#####{}. {}'.format(file_num, file))
        infodict = fullinfo(file)
        _head = np.zeros((6, 1))
        
        test_tree = parse(path + file)
        test_element = test_tree.documentElement
        #***八度变化***
        octavechangeNodes = test_element.getElementsByTagName('octave-change')
        if len(octavechangeNodes) != 0:
            octavechange = int(octavechangeNodes[0].firstChild.data)
        else:
            octavechange = 0
            
        #***capo****
        capo = 0
        if haveNodes(test_element, 'capo'):
            capo = int(test_element.getElementsByTagName('capo')[0].firstChild.data)
            gl_capo = capo
            print('capo {}'.format(capo))
            
        #***记录调弦信息, 特殊调弦处理***#***
        staff_tuning = test_element.getElementsByTagName('staff-tuning')
        tunings = [0, 0, 0, 0, 0, 0]
        for Node in staff_tuning:
            linenum = Node.getAttribute('line')
            tuning_octave = Node.getElementsByTagName('tuning-octave')[0].firstChild.data
            tuning_step = Node.getElementsByTagName('tuning-step')[0].firstChild.data
            tuning_alter = '0'
            if haveNodes(Node, 'tuning-alter'):
                tuning_alter = Node.getElementsByTagName('tuning-alter')[0].firstChild.data
            #print(tuning_octave + tuning_step + tuning_alter)
            tunings[int(linenum) - 1] = (str2midi(tuning_octave + tuning_step) + int(tuning_alter))
           
        standard = [40, 45, 50, 55, 59, 64]
        current = [i[1] - i[0] for i in zip(standard, tunings)]
        gl_tune = [i+12*octavechange for i in current]
        if show_tuning:
            print('special tuning:{}, octave-change:{}'.format(gl_tune, octavechange))
                    
        measures = test_element.getElementsByTagName('measure') #获取所有小节的nodes
        beats = getAttr(measures[0], 'beats')  #拍号
        beat_type = getAttr(measures[0], 'beat-type')
        timesig = []
        pitchs = []
        timesigs = []
        fingers = []
    
        repeatmark, housemark = check_repeat(measures)
        
        for measure_num, measure in enumerate(measures):
            if show_processing == True:
                print('measure {} processing...'.format(measure_num))
            #******获得divisions******
            if haveNodes(measure, 'divisions'):
                divisions = measure.getElementsByTagName('divisions')[0].firstChild.data[0]
                divisions = int(divisions)
                
            parts = get_parts(measure) 
            partdicts = []
            for part_num, part in enumerate(parts):
                if show_processing == True:
                    print('  part {} processing...'.format(part_num))
                partdict = {
                  'dictpitch':0, 
                  'dictfinger':0, 
                  'dicttime':0
                }
                #__timesig, __pitch, __finger = tie_clean(part, divisions=divisions, beats=beats)
                partdict['dicttime'], partdict['dictpitch'], partdict['dictfinger'] = tie_clean(part, divisions=divisions, beats=beats)
                partdicts.append(partdict)
                
            _pitch, _finger, _total_times = part_integrate(partdicts)
            _total_times.append(int(beats))
            _timesig = list(np.diff(np.array(_total_times)))
            # print('timesig:{}'.format(timesig))
            # print('pitch:{}'.format(pitch))
            # print('finger:{}'.format(finger))
            should = float(beats) / (float(beat_type)//4)
            try:
                assert abs(sum(_timesig) - should) / should <= 0.05
            except:
                print('    ! In measure {}, sum of timesig should be {}, but{}'.format(measure_num, should, sum(_timesig)))
                                 
            assert len(_timesig)==len(_pitch)
            assert len(_timesig)==len(_finger)

            for i in range(len(_timesig)):
                _pitch[i] = str_postprocess(_pitch[i], pitchtype=pitchtype)
                _finger[i] = str_postprocess(_finger[i], mode='finger')
            timesigs.append(_timesig)
            pitchs.append(_pitch)
            fingers.append(_finger)
        
        #根据反复记号的mark来重复写一些小节
        #print('repeatmark: {}'.format(repeatmark))
        #print('housemark: {}'.format(housemark))
        pitchs = repeater(pitchs, repeatmark, housemark)
        fingers = repeater(fingers, repeatmark, housemark)
        timesigs = repeater(timesigs, repeatmark, housemark)
        #print(pitchs)     
        info = TABinfo(capo=capo, tuning=gl_tune, original=infodict['original'], name=infodict['name'], beat=beats, beat_type=beat_type)
        tab = Tablature(pitch=pitchs, finger=fingers, time=timesigs, info=info)
        
        TABobjects.append(tab)
    return TABobjects

def measure_join(measurelist):
    '''
    join many measure into one Tablature object
    '''
    pitchlist, timelist, fingerlist = [], [], []
    for measure in measurelist:
        pitchlist.append(measure.pitch)
        fingerlist.append(measure.finger)
        timelist.append(measure.time)
    
    return1 = Tablature(pitch=pitchlist, finger=fingerlist, time=timelist)
    return return1

def clipinfo(clipname):
    clipname = clipname[:-4]
    structure = clipname[-1]
    
    startbar = int(clipname[clipname.index('_')+1:clipname.index('^')])
    endbar = int(clipname[clipname.index('^')+1:-1])
    originals = clipname[clipname.index('[')+1:clipname.index(']')]
    name = clipname[clipname.index(']')+1:clipname.index('_')]
    infodict = {
        'originals': originals,
        'name': name,
        'startbar': startbar,
        'endbar': endbar     
    }
    return infodict
def fullinfo(fullname):
    index = fullname.split(']')
    original = index[0][1:]
    name = index[-1]
    if len(index) == 1:
        original=None
    infodict = {
        'original': original,
        'name': name[:-4],  
    }
    return infodict

In [130]:
a = [1,2,3,4]
b = [5,6,7,8]
c = [9, 0, 1,2]
measure1 = Tablature(pitch=a, finger=b, time=c)
measure2 = Tablature(pitch=b, finger=c, time=a)
measure_join([measure1, measure2]).pitch

[[1, 2, 3, 4], [5, 6, 7, 8]]

In [131]:
path = os.getcwd()
songs = readTAB(path)

#####0. Empty.xml
#####1. [Air]鸟之诗_10^27B.xml


In [132]:
birdsong = songs[1]
for i in birdsong.vectorization().pitch[:3]:
    print(i)

['4E 2G', '__', '__', '__', '4E', '__', '4D 2G', '4E 2G', '__', '__', '3D', '__', '3A', '__', '3B', '__']
['4C 2A', '__', '3B', '__', '4D 2A', '__', '4E 2A', '__', '4G 3D', '__', '4E', '__', '3B', '__', '4C', '__']
['3B 2A', '__', '__', '__', '3B', '__', '3A 2A', '3E 2A', '__', '__', '2F#', '__', '2A', '__', '__', '__']


In [133]:
root_melody = birdsong.root_and_melody()
root_melody.pitch

[['4E 2G', '4E', '4D 2G', '4E 2G', '3A', '3B'],
 ['4C 2A', '3B', '4D 2A', '4E 2A', '4G', '4E', '3B', '4C'],
 ['3B 2A', '3B', '3A 2A', '3E 2A', '2F#', '2A'],
 ['4A', '4E', '3A', '3B', '4C', '4G'],
 ['4E 2G', '4E', '4D 2G', '4E 2G', '3A', '3B'],
 ['4C 2A', '3B', '4D 2A', '4E 2A', '4G', '4E', '4G 2E', '5C'],
 ['4B 2A', '4A', '3G', '4E', '2A', '2B', '4D'],
 ['4E', '4G', '4A', '4B'],
 ['4E 2G', '4E', '4D 2G', '4E 2G', '3A', '3B'],
 ['4C 2A', '3B', '4D 2A', '4E 2A', '4G', '4E', '3B', '4C'],
 ['3B 2A', '3B', '3A', '2F#', '2A'],
 ['3G', '4E', '3B', '3A', '3B', '4C', '4G'],
 ['4E 2G', '4E', '4D 2G', '4E 2G', '3A', '3B'],
 ['4C 2A', '3B', '4D 2A', '4E 2A', '4G', '4E', '4G 2E', '5C'],
 ['4B 2A', '3G', '4C', '4E', '3B', '3G', '2B'],
 ['4A 3C', '4C', '3B', '4C', '4D', '4E', '4A', '5C'],
 ['4B 2A', '3G', '4C', '4E', '3B', '3G', '3D', '3B'],
 ['4A 2A', '3G', '3B', '4C', '4D', '3B', '3G']]

In [77]:

testmeasure = songs[1].measure(0)
pitchvec, fingervec = vectorization(testmeasure)
print(pitchvec)

['4E 2G', '__', '__', '__', '4E', '__', '4D 2G', '4E 2G', '__', '__', '3D', '__', '3A', '__', '3B', '__']


In [31]:

path = os.getcwd()
songs = readTAB(path)

#####0. Empty.xml
[['2E', '2F', '2F#', '2G'], ['2A', '2A#', '2B', '3C'], ['2E', '2F', '2F#', '2G'], ['2A', '2A#', '2B', '3C'], ['2E', '2F', '2F#', '2G'], ['3D', '3D#', '3E', '3F']]
!!!
0


In [43]:
songs[1].time

[[1.0, 0.5, 0.25, 0.75, 0.5, 0.5, 0.5],
 [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5],
 [1.0, 0.5, 0.25, 0.75, 0.5, 1.0],
 [0.5, 1.0, 1.0, 0.5, 0.5, 0.5],
 [1.0, 0.5, 0.25, 0.75, 0.5, 0.5, 0.5],
 [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5],
 [0.75, 0.25, 0.5, 1.0, 0.5, 0.5, 0.5],
 [1.0, 1.0, 1.0, 1.0],
 [1.0, 0.5, 0.25, 0.75, 0.5, 0.5, 0.5],
 [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5],
 [0.5, 0.5, 0.5, 0.25, 0.75, 0.5, 0.5, 0.5],
 [0.5, 0.5, 0.5, 1.0, 0.5, 0.5, 0.5],
 [1.0, 0.5, 0.25, 0.75, 0.5, 0.5, 0.5],
 [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5],
 [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5],
 [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5],
 [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5],
 [0.5, 0.5, 0.25, 0.25, 1.0, 0.5, 1.0]]

In [45]:
melody1 = root_detect(measure1_3)
melody1.pitch

['2E', '2F', '2F#', '2G']

In [ ]:
pitchs = []
pitchs.append(['OOOOOO'])
pitchs.append(['PPPPPP'])
pitchs.append(['QQQQQQ'])


repeatmarks = check_repeat(measures)
print(repeatmarks)
repeatpairs = []
for marknum, mark in enumerate(repeatmarks):
    if 'a' in mark:
        start = marknum
    if '_' in mark:
        housestart = marknum
    if 'b' in mark:
        end = marknum
        times = int(mark[-1])
        repeatpairs.append((start, housestart, end, housetimes, times))
        

In [ ]:
repeatpairs

In [28]:
class ob1:
    def __init__(self, arg1=111):
        self.arg1 = arg1
        
    def measure(self):
        a = ob1(arg1=2222)
        print(a.arg1)
        return a
        
        

In [29]:
q = ob1()
x = q.measure()
x.arg1

2222


2222

In [ ]:
pitchs = [['小节1'], ['小节2'], ['小节3'], ['小节4'], ['小节5'], ['小节6']]
repeatmark = ['a', 'b3', 0, 'a', 'b3', 0]
housemark = [0, '12ab', '3a', 0, '12ab', '3a']

play = []

repeater(pitchs, repeatmark, housemark)

In [ ]:
p, f, t, tune = readTAB(os.getcwd()+'\\')
check_repeat(measures)
play = repeater(p[0], repeatmark, housemark)
for bar in play:
    print(bar)
    

In [47]:
a = [1,2,3]
b = [4,5,6]
c = [a, b]
d = []
for q in c:
    d.append([x for x in q])
    
d

In [14]:
a = ['','','','','','','','']
b = [1, '', '', '']
if len([i for i in a if i != '']) != 0:
    print('not an empty')
else:
    print('empty')

empty
